In [10]:
import pandas as pd 
import numpy as np
import requests
import csv
import time

In [3]:
url = "http://localhost:3000/api/games"

In [38]:
blue_params = {'gameId': 21738, 'side': 'Blue'}
red_params = {'gameId': 21738, 'side': 'Red'}
blue_response = requests.get(url, params=blue_params)
red_response = requests.get(url, params=red_params)

In [39]:
blue_response.json()

[{'id': 21738,
  'match_id': 13351,
  'game_number': 1,
  'blue_team': 1655,
  'red_team': 1641,
  'blue_win': 1,
  'duration': 2162,
  'game_id': 21738,
  'side': 'Blue',
  'first_blood': 0,
  'first_tower': 0,
  'towers': 11,
  'tower_plates': 0,
  'void_grubs': 0,
  'rift_heralds': 0,
  'baron_nashors': 2,
  'infernals': 1,
  'mountains': 1,
  'clouds': 0,
  'oceans': 0,
  'hextechs': 0,
  'chemtechs': 0,
  'elders': 0,
  'feats_of_strength': 0,
  'ruinous_atakhan': 0,
  'voracious_atakhan': 0}]

In [44]:
df = pd.DataFrame(blue_response.json())
df[['blue_win', 'game_number', 'duration', 'first_blood', 'first_tower', 'towers', 'tower_plates', 'void_grubs', 'rift_heralds',
    'baron_nashors', 'infernals', 'mountains', 'clouds', 'oceans', 'hextechs', 'chemtechs', 'elders', 'feats_of_strength',
    'ruinous_atakhan', 'voracious_atakhan']]

,blue_win,game_number,duration,first_blood,first_tower,towers,tower_plates,void_grubs,rift_heralds,baron_nashors,infernals,mountains,clouds,oceans,hextechs,chemtechs,elders,feats_of_strength,ruinous_atakhan,voracious_atakhan
0,1,1,2162,0,0,11,0,0,0,2,1,1,0,0,0,0,0,0,0,0


In [45]:
df = pd.DataFrame(red_response.json())
df[['first_blood', 'first_tower', 'towers', 'tower_plates', 'void_grubs', 'rift_heralds',
    'baron_nashors', 'infernals', 'mountains', 'clouds', 'oceans', 'hextechs', 'chemtechs', 'elders', 'feats_of_strength',
    'ruinous_atakhan', 'voracious_atakhan']]

,first_blood,first_tower,towers,tower_plates,void_grubs,rift_heralds,baron_nashors,infernals,mountains,clouds,oceans,hextechs,chemtechs,elders,feats_of_strength,ruinous_atakhan,voracious_atakhan
0,1,1,3,0,0,2,0,0,2,0,1,0,0,0,0,0,0


In [46]:
url = "http://localhost:3000/api/playerperformances"
blue_params = {'gameId': 21738, 'side': 'Blue'}
red_params = {'gameId': 21738, 'side': 'Red'}
blue_response = requests.get(url, params=blue_params)
red_response = requests.get(url, params=red_params)

In [51]:
df = pd.DataFrame(blue_response.json())
df = df[['role', 'kills', 'deaths', 'assists', 'damage_to_champions', 'wards_placed', 'wards_destroyed',
    'control_wards_bought', 'vision_score', 'total_gold', 'gold_spent', 'creep_score', 
    'kills_at_15', 'deaths_at_15', 'assists_at_15', 'gold_at_15']]
df_encoded = pd.get_dummies(df, columns=['role'], drop_first=False)
df_encoded

vector = df_encoded.values.flatten()
vector

array([3, 3, 3, -1, -1, -1, -1, -1, 13052, -1, -1, -1, -1, -1, -1, False,
       False, False, False, True, 1, 0, 12, -1, -1, -1, -1, -1, 12383, -1,
       -1, -1, -1, -1, -1, False, True, False, False, False, 2, 1, 9, -1,
       -1, -1, -1, -1, 11470, -1, -1, -1, -1, -1, -1, False, False, True,
       False, False, 5, 0, 8, -1, -1, -1, -1, -1, 18252, -1, -1, -1, -1,
       -1, -1, True, False, False, False, False, 2, 0, 11, -1, -1, -1, -1,
       -1, 9135, -1, -1, -1, -1, -1, -1, False, False, False, True, False],
      dtype=object)

In [2]:
def game_vector(gameId):
    objectives = "http://localhost:3000/api/games"
    player_stats = "http://localhost:3000/api/playerperformances"

    blue_params = {'gameId': gameId, 'side': 'Blue'}
    red_params = {'gameId': gameId, 'side': 'Red'}
    blue_objectives = requests.get(objectives, params=blue_params)
    red_objectives = requests.get(objectives, params=red_params)
    blue_player_stats = requests.get(player_stats, params=blue_params)
    red_player_stats = requests.get(player_stats, params=red_params)

    blue_objectives_df = pd.DataFrame(blue_objectives.json())
    blue_objectives_df = blue_objectives_df[['blue_win', 'game_number', 'duration', 'first_blood', 'first_tower', 'towers', 'tower_plates', 'void_grubs', 'rift_heralds',
    'baron_nashors', 'infernals', 'mountains', 'clouds', 'oceans', 'hextechs', 'chemtechs', 'elders', 'feats_of_strength',
    'ruinous_atakhan', 'voracious_atakhan']]

    vector = blue_objectives_df.values.flatten().tolist()

    blue_player_stats_df = pd.DataFrame(blue_player_stats.json())
    blue_player_stats_df = blue_player_stats_df[['role', 'kills', 'deaths', 'assists', 'damage_to_champions', 'wards_placed', 'wards_destroyed',
        'control_wards_bought', 'vision_score', 'total_gold', 'gold_spent', 'creep_score', 
        'kills_at_15', 'deaths_at_15', 'assists_at_15', 'gold_at_15']].sort_values(by='role')
    blue_player_stats_df_encoded = pd.get_dummies(blue_player_stats_df, columns=['role'], drop_first=False)

    vector += blue_player_stats_df_encoded.values.flatten().tolist()

    red_objectives_df = pd.DataFrame(red_objectives.json())
    red_objectives_df = red_objectives_df[['first_blood', 'first_tower', 'towers', 'tower_plates', 'void_grubs', 'rift_heralds',
    'baron_nashors', 'infernals', 'mountains', 'clouds', 'oceans', 'hextechs', 'chemtechs', 'elders', 'feats_of_strength',
    'ruinous_atakhan', 'voracious_atakhan']]

    vector += red_objectives_df.values.flatten().tolist()

    red_player_stats_df = pd.DataFrame(red_player_stats.json())
    red_player_stats_df = red_player_stats_df[['role', 'kills', 'deaths', 'assists', 'damage_to_champions', 'wards_placed', 'wards_destroyed',
        'control_wards_bought', 'vision_score', 'total_gold', 'gold_spent', 'creep_score', 
        'kills_at_15', 'deaths_at_15', 'assists_at_15', 'gold_at_15']].sort_values(by='role')
    red_player_stats_df_encoded = pd.get_dummies(red_player_stats_df, columns=['role'], drop_first=False)

    vector += red_player_stats_df_encoded.values.flatten().tolist()

    return vector

In [5]:
game_vector(21738)

[1,
 1,
 2162,
 0,
 0,
 11,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 8,
 -1,
 -1,
 -1,
 -1,
 -1,
 18252,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 True,
 False,
 False,
 False,
 False,
 1,
 0,
 12,
 -1,
 -1,
 -1,
 -1,
 -1,
 12383,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 False,
 True,
 False,
 False,
 False,
 2,
 1,
 9,
 -1,
 -1,
 -1,
 -1,
 -1,
 11470,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 False,
 False,
 True,
 False,
 False,
 2,
 0,
 11,
 -1,
 -1,
 -1,
 -1,
 -1,
 9135,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 False,
 False,
 False,
 True,
 False,
 3,
 3,
 3,
 -1,
 -1,
 -1,
 -1,
 -1,
 13052,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 False,
 False,
 False,
 False,
 True,
 1,
 1,
 3,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 13839,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 True,
 False,
 False,
 False,
 False,
 2,
 2,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 10718,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 False,
 True,
 False,
 False,
 False,
 1,
 2,
 3,
 -1,
 -1,
 -1,
 -1,
 -1,
 12101,
 -1,

In [4]:
len(game_vector(21738))

237

In [5]:
vectors = []
for i in range(1, 57203):
    vector = game_vector(i)
    print(f"Game {i}")
    vectors.append(vector)
    time.sleep(0.1)

Game 1
Game 2
Game 3
Game 4
Game 5
Game 6
Game 7
Game 8
Game 9
Game 10
Game 11
Game 12
Game 13
Game 14


KeyboardInterrupt: 

In [64]:
file_path = 'data.csv'

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(vectors)

print(f"Data written to {file_path}")

Data written to data.csv


In [3]:
def team_game_vector(gameId, teamId):
    objectives = "http://localhost:3000/api/games"
    player_stats = "http://localhost:3000/api/playerperformances"

    blue_params = {'gameId': gameId, 'side': 'Blue'}
    red_params = {'gameId': gameId, 'side': 'Red'}
    blue_objectives = requests.get(objectives, params=blue_params)
    red_objectives = requests.get(objectives, params=red_params)
    blue_player_stats = requests.get(player_stats, params=blue_params)
    red_player_stats = requests.get(player_stats, params=red_params)

    blue_objectives_df = pd.DataFrame(blue_objectives.json())

    teamBlueSide = (blue_objectives_df['blue_team'].iloc[0] == teamId)

    blue_objectives_df = blue_objectives_df[['blue_win', 'game_number', 'duration', 'first_blood', 'first_tower', 'towers', 'tower_plates', 'void_grubs', 'rift_heralds',
    'baron_nashors', 'infernals', 'mountains', 'clouds', 'oceans', 'hextechs', 'chemtechs', 'elders', 'feats_of_strength',
    'ruinous_atakhan', 'voracious_atakhan']]
    blue_objectives_df = blue_objectives_df.astype({col: int for col in blue_objectives_df.columns if blue_objectives_df[col].dtype == 'bool'})
    blue_objectives_df_values = blue_objectives_df.values.flatten().tolist()

    blue_player_stats_df = pd.DataFrame(blue_player_stats.json())
    blue_player_stats_df = blue_player_stats_df[['role', 'kills', 'deaths', 'assists', 'damage_to_champions', 'wards_placed', 'wards_destroyed',
        'control_wards_bought', 'vision_score', 'total_gold', 'gold_spent', 'creep_score', 
        'kills_at_15', 'deaths_at_15', 'assists_at_15', 'gold_at_15']].sort_values(by='role')
    blue_player_stats_df = blue_player_stats_df.astype({col: int for col in blue_player_stats_df.columns if blue_player_stats_df[col].dtype == 'bool'})
    blue_player_stats_df_encoded = pd.get_dummies(blue_player_stats_df, columns=['role'], drop_first=False)

    blue_player_stats_df_encoded_values = blue_player_stats_df_encoded.values.flatten().tolist()

    red_objectives_df = pd.DataFrame(red_objectives.json())
    red_objectives_df = red_objectives_df[['first_blood', 'first_tower', 'towers', 'tower_plates', 'void_grubs', 'rift_heralds',
    'baron_nashors', 'infernals', 'mountains', 'clouds', 'oceans', 'hextechs', 'chemtechs', 'elders', 'feats_of_strength',
    'ruinous_atakhan', 'voracious_atakhan']]
    red_objectives_df = red_objectives_df.astype({col: int for col in red_objectives_df.columns if red_objectives_df[col].dtype == 'bool'})
    red_objectives_df_values =  red_objectives_df.values.flatten().tolist()

    red_player_stats_df = pd.DataFrame(red_player_stats.json())
    red_player_stats_df = red_player_stats_df[['role', 'kills', 'deaths', 'assists', 'damage_to_champions', 'wards_placed', 'wards_destroyed',
        'control_wards_bought', 'vision_score', 'total_gold', 'gold_spent', 'creep_score', 
        'kills_at_15', 'deaths_at_15', 'assists_at_15', 'gold_at_15']].sort_values(by='role')
    red_player_stats_df = red_player_stats_df.astype({col: int for col in red_player_stats_df.columns if red_player_stats_df[col].dtype == 'bool'})
    red_player_stats_df_encoded = pd.get_dummies(red_player_stats_df, columns=['role'], drop_first=False)

    red_player_stats_df_encoded_values = red_player_stats_df_encoded.values.flatten().tolist()

    if teamBlueSide:
        vector = blue_objectives_df_values + blue_player_stats_df_encoded_values + red_objectives_df_values + red_player_stats_df_encoded_values
    else:
        vector = red_objectives_df_values + red_player_stats_df_encoded_values + blue_objectives_df_values + blue_player_stats_df_encoded_values

    return vector

In [4]:
def match_vector(teamId, matchId):
    games = "http://localhost:3000/api/games"

    game_params = {'matchId': matchId}

    game_response = requests.get(games, params=game_params)
    game_response_df = pd.DataFrame(game_response.json())

    # print(game_response_df)

    game_ids = game_response_df['id'].tolist()
    vectors = []
    for gameId in game_ids:
        vector = team_game_vector(gameId, teamId)

        if len(vector) == 237:
            vectors.append(vector)
        time.sleep(0.1)

    vectors_array = np.array(vectors)

    # print(vectors_array.shape)

    if len(vectors) > 0:
        vectors_array = np.array(vectors)
        average_vector = vectors_array.mean(axis=0)
        average_vector[0] = round(average_vector[0])
        return average_vector
    else:
        print(f"No valid vectors for matchId {matchId}") 
        return []  

In [31]:
all_examples = []

In [34]:
# all_examples = []
# for i in range(1, 10): #4144
for i in range(3705, 4144): #4144 #3703
    print(i)
    matches = "http://localhost:3000/api/matches"
    team_params = {'teamId': i}

    team_matches = requests.get(matches, params=team_params)
    team_matches_df = pd.DataFrame(team_matches.json())

    if team_matches_df.empty:
        continue

    team_matches_ids = team_matches_df['id'].tolist() 

    if len(team_matches_ids) < 11:
        continue 

    print(team_matches_ids)
    team_match_vectors = [match_vector(i, matchId) for matchId in team_matches_ids]
    team_match_vectors = [vec for vec in team_match_vectors if len(vec) > 0]
    if len(team_match_vectors) < 11:
        continue

    context_length = 10
    for j in range(len(team_match_vectors) - context_length):
        past_vectors = team_match_vectors[j : j + context_length]
        future_vector = team_match_vectors[j + context_length]

        example = {
            "past_values": [vec.tolist() for vec in past_vectors],
            "future_values": [future_vector.tolist()],
        }

        all_examples.append(example)

3705
[35863, 35871, 35874, 35881, 35883, 35884, 35892, 35894, 35902, 35909, 35913, 35916, 35917]
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
[36400, 36404, 36407, 36637, 36640, 36647, 36648, 36654, 36658, 36661, 36664, 36671, 36675, 36676, 37402, 37407, 37412, 37417, 37421, 37424]
3730
3731
3732
[36400, 36402, 36409, 36643, 36645, 36650, 36654, 36656, 36663, 36667, 36668, 36673, 36678, 36681, 37405, 37406, 37410, 37416, 37419, 37425]
3733
3734
3735
[36428, 36429, 36433, 36735, 36738, 36739, 36740, 37496, 37498, 37501, 37504, 37506, 37511]
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
[36289, 36295, 36301, 36302, 36308, 36312, 36521, 36526, 36530, 36536, 36538, 36544, 36546, 37643, 37650, 37655, 37662, 37666, 37672]
3754
[36287, 36296, 36301, 36303, 36309, 36313, 36520, 36524, 36529, 37645, 37649, 37657, 37662, 37663, 37668]
3755
3756
3757
3758
3759
3760
[36415, 36418,

KeyError: "None of [Index(['role', 'kills', 'deaths', 'assists', 'damage_to_champions',\n       'wards_placed', 'wards_destroyed', 'control_wards_bought',\n       'vision_score', 'total_gold', 'gold_spent', 'creep_score',\n       'kills_at_15', 'deaths_at_15', 'assists_at_15', 'gold_at_15'],\n      dtype='object')] are in the [columns]"

In [ ]:
import json

# Save the examples to a .jsonl file
with open("team_match_dataset2.jsonl", "w") as f:
    for example in all_examples:
        # Write each example as a JSON line
        f.write(json.dumps(example) + "\n")

In [ ]:
# To Do

# generate all match data with who won 

# look at which teams played and get their last ten matches
    # if either have less than 10 matches skip 

# the pipeline evalutation dataset needs to look like this:
# include blue team id, red team id, the date of the match, and who won 
# the model pipeline will gather the last ten matches for each team prior to a date for us 
# find accuracy for overall model 

# modify the app predict to take in a date
# write a new api call to get the last ten matches for a team prior to a certain date 
# use those ten matches to generate a vector for prediction for a match on a certain date 

In [14]:
matches = pd.read_csv("MatchTeamsDateWinner2025.csv")
matches.head()

,id,team_one,team_two,date,blueWin
0,36244,3763,3850,2025-01-11,0
1,36245,3851,3852,2025-01-11,1
2,36246,3853,3854,2025-01-11,0
3,36247,3855,3856,2025-01-11,0
4,36248,3850,3853,2025-01-12,1


In [15]:
validMatches = []

In [23]:
def validTeamForMatch(teamId, date) -> bool:
    matches = "http://localhost:3000/api/matches"
    team_params = {'teamId': teamId, 'gamesPlayed': 10, 'date': date}
    team_matches = requests.get(matches, params=team_params)
    team_matches_df = pd.DataFrame(team_matches.json())

    try:
        if team_matches_df.empty:
            return False

        team_matches_ids = team_matches_df['id'].tolist() 

        if len(team_matches_ids) < 10:
            return False

        # print(team_matches_ids)
        team_match_vectors = [match_vector(i, matchId) for matchId in team_matches_ids]
        team_match_vectors = [vec for vec in team_match_vectors if len(vec) > 0]
        if len(team_match_vectors) < 10:
            return False
        
        return True
    except Exception as e:
        return False

In [25]:
for i, row in matches.iterrows():
    if validTeamForMatch(row['team_one'], row['date']) and validTeamForMatch(row['team_two'], row['date']):
        validMatches.append(i)
        print(f"Valid match found: {row['id']} {row['team_one']} vs {row['team_two']} on {row['date']}")
    else:
        print(f"Invalid match: {row['id']} {row['team_one']} vs {row['team_two']} on {row['date']}")
    time.sleep(0.1)

Invalid match: 36244 3763 vs 3850 on 2025-01-11
Invalid match: 36245 3851 vs 3852 on 2025-01-11
Invalid match: 36246 3853 vs 3854 on 2025-01-11
Invalid match: 36247 3855 vs 3856 on 2025-01-11
Invalid match: 36248 3850 vs 3853 on 2025-01-12
Invalid match: 36249 3763 vs 3852 on 2025-01-12
Invalid match: 36250 3851 vs 3855 on 2025-01-12
Invalid match: 36251 3854 vs 3856 on 2025-01-12
Invalid match: 36252 3852 vs 3855 on 2025-01-18
Invalid match: 36253 3851 vs 3854 on 2025-01-18
Invalid match: 36254 3853 vs 3763 on 2025-01-18
Invalid match: 36255 3850 vs 3856 on 2025-01-18
Invalid match: 36256 3852 vs 3854 on 2025-01-19
Invalid match: 36257 3850 vs 3851 on 2025-01-19
Invalid match: 36258 3855 vs 3763 on 2025-01-19
Invalid match: 36259 3853 vs 3856 on 2025-01-19
Invalid match: 36260 3763 vs 3856 on 2025-01-25
Invalid match: 36261 3850 vs 3852 on 2025-01-25
Invalid match: 36262 3853 vs 3851 on 2025-01-25
Invalid match: 36263 3856 vs 3851 on 2025-01-26
Invalid match: 36264 3853 vs 3852 on 202

In [26]:
validMatches

[249,
 253,
 255,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 420,
 421,
 422,
 424,
 425,
 426,
 427,
 428,
 429,
 430,
 431,
 432,
 433,
 434,
 435,
 436,
 437,
 440,
 441,
 442,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 660,
 662,
 664,
 667,
 670,
 672,
 673,
 679,
 689,
 697,
 704,
 872,
 877,
 878,
 880,
 885,
 887,
 888,
 890,
 891,
 892,
 893,
 958,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1087,
 1089,
 1090,
 1091,
 1092,
 1093,
 1095,
 1096,
 1097,
 1098,
 1099,
 1100,
 1101,
 1102,
 1103,
 1105,
 1106,
 1107,
 1108,
 1110,
 1111,
 1112,
 1113,
 1114,
 1115,
 1116,
 1117,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125,
 11

In [28]:
filtered_matches = matches.iloc[validMatches]
filtered_matches.to_csv("FilteredMatchTeamsDateWinner2025.csv", index=False)